# Merging the Datasets

Prior to this step we have  downloaded a number of economic datasets from https://fred.stlouisfed.org/ including informationon commodities, consumer sentiment, external trade, growth and activity, housing and construction, inflation, interest, labor, market sentiment, and monetary conditions. Additionally we have downloaded historical data on the 8 major foreign currencie pairs and the SPY index. 

In these step we will be combining data into one single file. 


#### Step 1: Merge Economic Data

In [ ]:
import sys; sys.path.append("..")
import os
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../../data/downloaded/economic_data")
OUT_CSV  = Path("../../data/generated/combined_economic_data.csv")

DATE_CANDIDATES = ["observation_date", "DATE", "date", "Date", "timestamp", "Timestamp"]
SUPPORTED = {".csv"}

panels = []

for root, _, files in os.walk(DATA_DIR):
    for fname in files:
        p = Path(root) / fname
        if p.suffix.lower() not in SUPPORTED:
            continue
        try:

            if p.suffix.lower() == ".csv":
                df = pd.read_csv(p)
            else:
                df = pd.read_excel(p, sheet_name=0)

            date_col = None
            for c in df.columns:
                if c in DATE_CANDIDATES:
                    date_col = c
                    break
            if date_col is None:
                for c in df.columns:
                    try:

                        pd.to_datetime(df[c], format="%Y-%m-%d", errors="coerce")

                        date_col = c
                        break
                    except Exception:
                        pass
            if date_col is None:
                print(f"Skipping (no date column): {p}")
                continue

            df = df.copy()
            df[date_col] = pd.to_datetime(df[date_col], format="%Y-%m-%d", errors="raise")
            df = df.dropna(subset=[date_col]).set_index(date_col).sort_index()
            df.index = pd.to_datetime(df.index).tz_localize(None).normalize()

            num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
            if not num_cols:
                print(f"Skipping (no numeric cols): {p}")
                continue
            df = df[num_cols]

            folder = Path(root).name or "root"
            base   = p.stem
            df.columns = [f"{folder}__{base}__{c}" for c in df.columns]

            panels.append(df)
            print(f"OK: {p} -> {len(df.columns)} cols, {df.shape[0]} rows")
        except Exception as e:
            print(f"Skipping (error): {p} -> {e}")

if not panels:
    raise RuntimeError("No usable CSV/Excel files found under econ_data")

merged = pd.concat(panels, axis=1, join="outer").sort_index()
merged.to_csv(OUT_CSV, index_label="date")
print(f"Saved {OUT_CSV} with shape {merged.shape}")


OK: ../../data/downloaded/economic_data/comodities/DHHNGSP.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/comodities/fredgraph.csv -> 2 cols, 61 rows
OK: ../../data/downloaded/economic_data/comodities/DCOILWTICO.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DGS10.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DGS2.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/CPN3M.csv -> 1 cols, 61 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DGS1.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DGS5.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DBAA.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/T10Y2Y.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data/interest_&_credit/DTB3.csv -> 1 cols, 1305 rows
OK: ../../data/downloaded/economic_data

#### Step 2: Merge Economic Data, Forex Data, and SPY Data

In [ ]:
from pathlib import Path
import pandas as pd

BASE      = Path("../../data/downloaded")
ECON_CSV  = Path("../../data/generated/combined_economic_data.csv")
FX_DIR    = BASE / "yfinance"
SPY_CSV   = BASE / "polygon" / "SPY.csv"
OUT_CSV   = Path("../../data/generated/fully_combined_finance_data.csv")

econ = pd.read_csv(ECON_CSV)
if "date" not in econ.columns and "Date" in econ.columns:
    econ = econ.rename(columns={"Date": "date"})
econ["date"] = pd.to_datetime(econ["date"], errors="coerce")
econ = econ.dropna(subset=["date"]).set_index("date").sort_index()
econ = econ[~econ.index.duplicated(keep="last")]

spy = pd.read_csv(SPY_CSV)
if "date" not in spy.columns and "Date" in spy.columns:
    spy = spy.rename(columns={"Date": "date"})
spy["date"] = pd.to_datetime(spy["date"], errors="coerce")
spy = spy.dropna(subset=["date"]).set_index("date").sort_index()
spy = spy[~spy.index.duplicated(keep="last")]

keep_cols = [c for c in spy.columns if c.lower() in {"open","high","low","close","volume","vwap","transactions"}]
if not keep_cols:
    keep_cols = [c for c in spy.columns if c.lower() in {"open","high","low","close","volume"}]
spy = spy[keep_cols].rename(columns={c: f"SPY_{c.lower()}" for c in keep_cols})

fx_frames = []
for p in sorted(FX_DIR.glob("*.csv")):
    df = pd.read_csv(p)
    if "date" not in df.columns and "Date" in df.columns:
        df = df.rename(columns={"Date": "date"})
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).set_index("date").sort_index()
    df = df[~df.index.duplicated(keep="last")]

    col = "close" if "close" in df.columns else ("Close" if "Close" in df.columns else None)
    if col is None:
        continue
    pair = p.stem.upper()
    fx_frames.append(df[[col]].rename(columns={col: f"FX_{pair}_close"}))

fx = pd.concat(fx_frames, axis=1, join="outer").sort_index() if fx_frames else pd.DataFrame()

merged = econ.join([spy, fx], how="outer").sort_index()

merged.to_csv(OUT_CSV, index_label="date")
print(f"Saved {OUT_CSV} with shape {merged.shape} "
      f"(econ cols: {econ.shape[1]}, SPY cols: {spy.shape[1]}, FX cols: {fx.shape[1] if not fx.empty else 0})")


Saved ../../data/generated/fully_combined_finance_data.csv with shape (7215, 74) (econ cols: 59, SPY cols: 7, FX cols: 8)
